In [75]:
import numpy as np
from tdmclient import ClientAsync, aw
import time

class Thymio():
    SENSORS_HORIZONTAL = "prox.horizontal"
    LEFT_MOTOR = "motor.left.target"
    RIGHT_MOTOR = "motor.right.target"
    
    LEDS_TOP = "leds.top"
    LEDS_BOTTOM_LEFT = "leds.bottom.left"
    LEDS_BOTTOM_RIGHT = "leds.bottom.right"
    LEDS_RC = "leds.rc"
    LEDS_TEMPERATURE = "leds.temperature"
    LEDS_PROX_H = "leds.prox.h"
    LEDS_PROX_V = "leds.prox.v"

    OBSTACLE_THRESHOLD = 1000
    SCALE = 0.01
    SPEED = 50

    def __init__(self):
        self.client = ClientAsync()
        self.node = aw(self.client.wait_for_node())
        aw(self.node.lock())

    def __del__(self):
        aw(self.node.unlock())
        
    def set_leds(self, leds: list, led_type: str):
        """
        Sets the LEDs of the Thymio
    
        :param leds: list of LED values
        :param led_type: type of LEDs to set
        """
    
        aw(self.node.set_variables({
            led_type: leds
        }))
        
    def reset_leds(self):
        """
        Resets the LEDs of the Thymio
        """
        
        self.set_leds([0, 0, 0], self.LEDS_TOP)
        self.set_leds([0, 0, 0], self.LEDS_BOTTOM_LEFT)
        self.set_leds([0, 0, 0], self.LEDS_BOTTOM_RIGHT)
        self.set_leds([0], self.LEDS_RC)
        self.set_leds([0, 0], self.LEDS_TEMPERATURE)
        self.set_leds([0, 0, 0, 0, 0, 0, 0, 0], self.LEDS_PROX_H)
        self.set_leds([0, 0], self.LEDS_PROX_V)

    def set_motors(self, left_motor: int, right_motor: int, verbose: bool = False):
        """
        Sets the motor speeds of the Thymio

        :param l_speed: left motor speed
        :param r_speed: right motor speed
        :param verbose: whether to print status messages or not
        """

        if verbose:
            print("\t\t Setting speed : ", left_motor, right_motor)

        aw(self.node.set_variables({
            self.LEFT_MOTOR: [left_motor],
            self.RIGHT_MOTOR: [right_motor]
        }))

    def get_horizontal_sensors(self, verbose: bool = False):
        """
        Returns the horizontal proximity sensors

        :param verbose: whether to print status messages or not
        """

        aw(self.client.wait_for_status(self.client.NODE_STATUS_READY))
        aw(self.node.wait_for_variables({self.SENSORS_HORIZONTAL}))

        values = self.node.var[self.SENSORS_HORIZONTAL]

        if verbose:
            print("\t\t Sensor values : ", values)

        return np.array(values)[:5]

    def local_navigation(self):
        """
        Local navigation until no obstacles are detected

        :return: True if no obstacles are detected
        """
        while np.any(self.get_horizontal_sensors() > self.OBSTACLE_THRESHOLD):
            W = np.array([[2,  1, -1, -1, -2], [-2, -1, -1,  1,  2]]) * self.SCALE

            motor_values = W @ self.get_horizontal_sensors().T + self.SPEED
            left_motor = int(motor_values[0])
            right_motor = int(motor_values[1])

            self.set_motors(left_motor, right_motor)

        return True

In [73]:
th = Thymio()

In [74]:
th.reset_leds()

In [9]:
for _ in range(100):
    print(th.get_horizontal_sensors())
    time.sleep(0.1)

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[   0 2238 4468 4043 3986]
[   0 2238 4468 4043 3986]
[   0 3394 4465 4048 4027]
[   0 3394 4465 4048 4027]
[   0    0 4381 3750 3275]
[   0    0 4381 3750 3275]
[   0    0 4381 3750 3275]
[   0    0    0 3162 2883]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[   0    0 2021 2166 2242]
[   0 4363 4196 3361 3183]
[   0 4363 4196 3361 3183]
[4082 2672 2904 2497    0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[   0    0 3253 1910 1023]
[   0 3226 4388 2652 1852]
[   0 3226 4388 2652 1852]
[   0 3226 4388 2652 1852]
[   0 2016 2935 1986 1662]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]


KeyboardInterrupt: 

In [8]:
th.local_navigation()

In [33]:
th.set_motors(0, 0)

In [76]:
th.__del__()